In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Mobile_Report

### Links:  

[[rep_mobile_citizens_id_city_partner]]

[[citizen_payments_st_mobile]]
___
### Table

In [5]:
query_text = """--sql
CREATE TABLE db1.t_mobile_report_rep_mobile_full
    (
        `report_date` Date,
        `partner_uuid` String,
        `city` String,
        `IOS_PL` Int64,
        `appstore_count_85` UInt64,
        `appstore_count_85_refunded` UInt64,
        `appstore_count_69` UInt64,
        `appstore_count_69_refunded` UInt64,
        `appstore_count_499` UInt64,
        `appstore_count_499_refunded` UInt64,
        `appstore_count_2390` UInt64,
        `appstore_count_2390_refunded` UInt64,
        `appstore_count_1` UInt64,
        `appstore_count_1_refunded` UInt64,
        `refunded_amount_appstore` Int64,
        `Android_PL` Int64,
        `yookassa_count_85` UInt64,
        `yookassa_count_85_refunded` UInt64,
        `yookassa_count_69` UInt64,
        `yookassa_count_69_refunded` UInt64,
        `yookassa_count_35` UInt64,
        `yookassa_count_35_refunded` UInt64,
        `yookassa_count_1` UInt64,
        `yookassa_count_1_refunded` UInt64,
        `yookassa_count_499` UInt64,
        `yookassa_count_499_refunded` UInt64,
        `yookassa_count_249` UInt64,
        `yookassa_count_249_refunded` UInt64,
        `yookassa_count_2390` UInt64,
        `yookassa_count_2390_refunded` UInt64,
        `refunded_amount_yookassa` Int64
    )
    ENGINE = MergeTree()
    ORDER BY report_date
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### Materialized view

In [6]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.t_mobile_report_rep_mobile_full_mv
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 40 MINUTES TO db1.t_mobile_report_rep_mobile_full AS
    SELECT
        report_date ,
        partner_uuid,
        city,
        ((`appstore_count_85` * 85 + `appstore_count_69` * 69 + `appstore_count_499` * 499 + `appstore_count_2390` * 2390 + `appstore_count_1` * 1) - `refunded_amount_appstore`)  as IOS_PL,
        --
        `appstore_count_85`,
        `appstore_count_85_refunded`,
        `appstore_count_69`,
        `appstore_count_69_refunded`,
        `appstore_count_499`,
        `appstore_count_499_refunded`,
        `appstore_count_2390`,
        `appstore_count_2390_refunded`,
        `appstore_count_1`,
        `appstore_count_1_refunded`,
        `refunded_amount_appstore`,
        --
        ((`yookassa_count_85` * 85 + `yookassa_count_69` * 69 + `yookassa_count_35` * 35 + `yookassa_count_1` * 1 + `yookassa_count_499` * 499 + `yookassa_count_249` * 249 +`yookassa_count_2390` * 2390) - `refunded_amount_yookassa`) as Android_PL,
        --
        `yookassa_count_85`,
        `yookassa_count_85_refunded`,
        `yookassa_count_69`, 
        `yookassa_count_69_refunded`,
        `yookassa_count_35`, 
        `yookassa_count_35_refunded`,
        `yookassa_count_1`, 
        `yookassa_count_1_refunded`, 
        `yookassa_count_499`, 
        `yookassa_count_499_refunded`,
        `yookassa_count_249`, 
        `yookassa_count_249_refunded`,  
        `yookassa_count_2390`, 
        `yookassa_count_2390_refunded`,  
        --
        `refunded_amount_yookassa`
    FROM    
        (SELECT
            citizen_payments_st_mobile_ch.report_date AS report_date,
            t_cit_id.partner_uuid  AS partner_uuid,
            t_cit_id.city AS city,
            count(if(`from` = 'appstore' and amount = 85 and citizen_payments_st_mobile.state = 'success', citizen_payments_st_mobile.citizen_id,null)) as appstore_count_85,
            count(if(`from` = 'appstore' and amount = 85 and citizen_payments_st_mobile.state = 'refunded', citizen_payments_st_mobile.citizen_id,null)) as appstore_count_85_refunded,
            count(if(`from` = 'appstore' and amount = 69 and citizen_payments_st_mobile.state = 'success', citizen_payments_st_mobile.citizen_id,null)) as appstore_count_69,
            count( if(`from` = 'appstore' and amount = 69 and citizen_payments_st_mobile.state = 'refunded', citizen_payments_st_mobile.citizen_id,null)) as appstore_count_69_refunded,
            count( if(`from` = 'appstore' and amount = 499 and citizen_payments_st_mobile.state = 'success', citizen_payments_st_mobile.citizen_id,null)) as appstore_count_499,
            count( if(`from` = 'appstore' and amount = 499 and citizen_payments_st_mobile.state = 'refunded', citizen_payments_st_mobile.citizen_id,null)) as appstore_count_499_refunded,
            count( if(`from` = 'appstore' and amount = 2390 and citizen_payments_st_mobile.state = 'success', citizen_payments_st_mobile.citizen_id,null)) as appstore_count_2390,
            count( if(`from` = 'appstore' and amount = 2390 and citizen_payments_st_mobile.state = 'refunded', citizen_payments_st_mobile.citizen_id,null)) as appstore_count_2390_refunded,
            count( if(`from` = 'appstore' and amount = 1 and citizen_payments_st_mobile.state = 'success', citizen_payments_st_mobile.citizen_id,null)) as appstore_count_1,
            count( if(`from` = 'appstore' and amount = 1 and citizen_payments_st_mobile.state = 'refunded', citizen_payments_st_mobile.citizen_id,null)) as appstore_count_1_refunded,
            --
            COALESCE(sum(if(`from` = 'appstore',`refunded_amount`,null)),0) as refunded_amount_appstore,
            --
            count( if(`from` = 'yookassa' and amount = 85 and citizen_payments_st_mobile.state = 'success', citizen_payments_st_mobile.citizen_id,null)) as yookassa_count_85,
            count( if(`from` = 'yookassa' and amount = 85 and citizen_payments_st_mobile.state = 'refunded', citizen_payments_st_mobile.citizen_id,null)) as yookassa_count_85_refunded,    
            count( if(`from` = 'yookassa' and amount = 69 and citizen_payments_st_mobile.state = 'success', citizen_payments_st_mobile.citizen_id,null)) as yookassa_count_69, 
            count( if(`from` = 'yookassa' and amount = 69 and citizen_payments_st_mobile.state = 'refunded', citizen_payments_st_mobile.citizen_id,null)) as yookassa_count_69_refunded,
            count( if(`from` = 'yookassa' and amount = 35 and citizen_payments_st_mobile.state = 'success', citizen_payments_st_mobile.citizen_id,null)) as yookassa_count_35, 
            count( if(`from` = 'yookassa' and amount = 35 and citizen_payments_st_mobile.state = 'refunded', citizen_payments_st_mobile.citizen_id,null)) as yookassa_count_35_refunded,
            count( if(`from` = 'yookassa' and amount = 1 and citizen_payments_st_mobile.state = 'success', citizen_payments_st_mobile.citizen_id,null)) as yookassa_count_1, 
            count( if(`from` = 'yookassa' and amount = 1 and citizen_payments_st_mobile.state = 'refunded', citizen_payments_st_mobile.citizen_id,null)) as yookassa_count_1_refunded, 
            count( if(`from` = 'yookassa' and amount = 499 and citizen_payments_st_mobile.state = 'success', citizen_payments_st_mobile.citizen_id,null)) as yookassa_count_499, 
            count( if(`from` = 'yookassa' and amount = 499 and citizen_payments_st_mobile.state = 'refunded', citizen_payments_st_mobile.citizen_id,null)) as yookassa_count_499_refunded,
            count( if(`from` = 'yookassa' and amount = 249 and citizen_payments_st_mobile.state = 'success', citizen_payments_st_mobile.citizen_id,null)) as yookassa_count_249, 
            count( if(`from` = 'yookassa' and amount = 249 and citizen_payments_st_mobile.state = 'refunded', citizen_payments_st_mobile.citizen_id,null)) as yookassa_count_249_refunded,  
            count( if(`from` = 'yookassa' and amount = 2390 and citizen_payments_st_mobile.state = 'success', citizen_payments_st_mobile.citizen_id,null)) as yookassa_count_2390, 
            count( if(`from` = 'yookassa' and amount = 2390 and citizen_payments_st_mobile.state = 'refunded', citizen_payments_st_mobile.citizen_id,null)) as yookassa_count_2390_refunded,  
            --
            COALESCE(sum(if(`from` = 'yookassa',refunded_amount,null)),0) as refunded_amount_yookassa,
            --
            max(citizen_payments_st_mobile.`report_date`) as max_report_date,
            min(citizen_payments_st_mobile.`report_date`) as min_report_date
        FROM db1.rep_mobile_citizens_id_city_partner AS t_cit_id
        ANY JOIN db1.`citizen_payments_st_mobile_ch` AS citizen_payments_st_mobile
        	ON citizen_payments_st_mobile.`report_date` = t_cit_id.`report_date`
        	AND citizen_payments_st_mobile.`citizen_id` = t_cit_id.`citizen_id`
        GROUP by 
            report_date,
            partner_uuid,
            city
            ) as t1
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [7]:
query_text = """--sql
    SELECT
        *
    FROM db1.t_mobile_report_rep_mobile_full_mv
    ORDER BY report_date desc
    limit 10
    """

ch.query_run(query_text)


report_date,partner_uuid,city,IOS_PL,appstore_count_85,appstore_count_85_refunded,appstore_count_69,appstore_count_69_refunded,appstore_count_499,appstore_count_499_refunded,appstore_count_2390,appstore_count_2390_refunded,appstore_count_1,appstore_count_1_refunded,refunded_amount_appstore,Android_PL,yookassa_count_85,yookassa_count_85_refunded,yookassa_count_69,yookassa_count_69_refunded,yookassa_count_35,yookassa_count_35_refunded,yookassa_count_1,yookassa_count_1_refunded,yookassa_count_499,yookassa_count_499_refunded,yookassa_count_249,yookassa_count_249_refunded,yookassa_count_2390,yookassa_count_2390_refunded,refunded_amount_yookassa
date,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
2025-06-05,"""a9ff7819-4e08-4113-94ba-ad06c3…","""Мытищи""",0,0,0,0,0,0,0,0,0,0,0,0,207,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0
2025-06-05,"""9143cc53-d22b-484b-ad94-0022f7…","""Санкт-Петербург""",0,0,0,0,0,0,0,0,0,0,0,0,346,0,0,5,0,0,0,1,0,0,0,0,0,0,0,0
2025-06-05,"""a478edca-2e04-4df3-afff-76fccf…","""деревня Глазково""",0,0,0,0,0,0,0,0,0,0,0,0,69,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2025-06-05,"""9b15a280-54fb-4ba6-8afe-31dfa3…","""Иваново""",0,0,0,0,0,0,0,0,0,0,0,0,69,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2025-06-05,"""e3c80fed-fc13-4c3f-a6df-af3a3f…","""Одинцово""",276,0,0,4,0,0,0,0,0,0,0,0,1449,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0
2025-06-05,"""6818fe8d-8f5e-4f93-a5d6-8e7ebd…","""посёлок Большое Исаково""",0,0,0,0,0,0,0,0,0,0,0,0,69,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2025-06-05,"""a1d713a3-5b0a-4f5d-aace-636c11…","""Белорецк""",0,0,0,0,0,0,0,0,0,0,0,0,208,0,0,3,0,0,0,1,0,0,0,0,0,0,0,0
2025-06-05,"""c0bdb685-1a35-45ca-8812-cd3db4…","""Ростов-на-Дону""",0,0,0,0,0,0,0,0,0,0,0,0,621,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0
2025-06-05,"""c259e6bf-517a-4f87-ba0c-99c7ea…","""Котельнич""",0,0,0,0,0,0,0,0,0,0,0,0,69,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.t_mobile_report_rep_mobile_full DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


### drop mv

In [4]:
query_text = """--sql
    DROP TABLE db1.t_mobile_report_rep_mobile_full_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop table

In [3]:
query_text = """--sql
    DROP TABLE db1.t_mobile_report_rep_mobile_full
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.t_mobile_report_rep_mobile_full_mv
"""

ch.query_run(query_text)